# Exploring API's

## Transaction data
I want to find all Events for one specific pump & dump event.

- [] Find event and timestamps
- [x] Find out on which chains it's running
- [x] Find out which ethereum blocks the timestamps map to
- [] Get data from ethereum blocks
- [] Filter data for specific coin
- [] Check Validity of data

## Price data

- [] find out price data for a specific coin and time range

In [45]:
import pandas as pd

## Find event and timestamps

In [46]:
pd_df = pd.read_csv("pump_telegram.csv")
counts = pd_df["symbol"].value_counts()
percs = pd_df["symbol"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])

,count,percentage
symbol,,
BRD,24,2.2%
RDN,24,2.2%
NXS,22,2.0%
BNT,22,2.0%
NAV,22,2.0%
...,...,...
DCRE,1,0.1%
DRACO,1,0.1%
IXT,1,0.1%


In [50]:
pd_df_rdn = pd_df[pd_df["symbol"] == "RDN"]
print(pd_df_rdn.head(1))

pd_df_brd = pd_df[pd_df["symbol"] == "BRD"]
print(pd_df_brd.head(1))

pd_df_nxs = pd_df[pd_df["symbol"] == "BNT"]
print(pd_df_nxs.head(1))

    symbol group        date   hour exchange
255    RDN   BPG  2018-08-21  16:00  binance
    symbol group        date   hour exchange
125    BRD   ATW  2018-12-22  17:00  binance
   symbol group        date   hour exchange
11    BNT   ATW  2018-01-13  16:00    yobit


In [48]:
import datetime

symbol = "rdn"
start_date = datetime.datetime.strptime("2018-08-21 16:00", "%Y-%m-%d %H:%M")

Open Questions:
How do timestamps align between data and different API's?

# Find out on which chains it's running


In [49]:
import requests

URL = "https://api.coingecko.com/api/v3/coins/list?include_platform=true"

r = requests.get(url = URL)
data = r.json()

for token in data:
    if token["symbol"] == symbol:
        print(token)
        eth_address = token["platforms"]["ethereum"]

TypeError: string indices must be integers, not 'str'

RDN runs on the eth blockchain

# Map date-range into block-range

In [ ]:
import time
from datetime import timedelta

# try to improve timezone problem
start_date = start_date + + timedelta(hours=4)

# convert to unix time
print("Given Date:", start_date)
unix_start_date = time.mktime(start_date.timetuple())
print("UNIX timestamp:", unix_start_date)

Given Date: 2018-08-21 20:00:00
UNIX timestamp: 1534874400.0


In [ ]:
# get block number

URL = f"https://coins.llama.fi/block/ethereum/{unix_start_date}"
r = requests.get(url = URL)
data = r.json()

print(data)
block_number = data["height"]

{'height': 6188705, 'timestamp': 1534874420}


Block number is 6187708

# Get transaction data for given date-range/block-range

### Alchemy API

In [ ]:
hex_block_number_start = hex(block_number)
hex_block_number_end = hex(block_number + 500)

In [ ]:
import json

alchemy_api_key = "4zWEsHMFRJ5CqkZLIp5m3322BRPbwB1q"
my_url = f"https://eth-mainnet.g.alchemy.com/v2/{alchemy_api_key}"
my_headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}
params = {'jsonrpc': '2.0', 'method': 'alchemy_getAssetTransfers', 'params': [
    {
      "fromBlock": hex_block_number_start,
      "toBlock": hex_block_number_end,
      "category": [
        "external",
        "internal",
        "erc20",
        "erc721",
        "erc1155",
        "specialnft"
      ],
      "withMetadata": False,
      "excludeZeroValue": True,
      "maxCount": "0x3e8" # maximum transactions possible
    }
  ]}


r = requests.post(url = my_url, json=params, headers=my_headers)
data = r.json()
json_data = json.dumps(data, indent=4)
 
with open("alchemy_answer.json", "w") as outfile:
    outfile.write(json_data)

json_data = json.loads(json_data)
print(len(json_data["result"]["transfers"]))

1000


In [ ]:
# with filter for eth-address

import json

alchemy_api_key = "4zWEsHMFRJ5CqkZLIp5m3322BRPbwB1q"
my_url = f"https://eth-mainnet.g.alchemy.com/v2/{alchemy_api_key}"
my_headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}
params = {'jsonrpc': '2.0', 'method': 'alchemy_getAssetTransfers', 'params': [
    {
      "fromBlock": hex_block_number_start,
      "toBlock": hex_block_number_end,
      "category": [
        "external",
        "internal",
        "erc20",
        "erc721",
        "erc1155",
        "specialnft"
      ],
      "withMetadata": False,
      "excludeZeroValue": True,
      "maxCount": "0x3e8" # maximum transactions possible
    }
  ]}


r = requests.post(url = my_url, json=params, headers=my_headers)
data = r.json()
json_data = json.dumps(data, indent=4)
 
with open("alchemy_answer_with_address_filter.json", "w") as outfile:
    outfile.write(json_data)

json_data = json.loads(json_data)
print(len(json_data["result"]["transfers"]))

1000


Open questions:
- How do I make sure I did get all transactions?
- What is the contact address field?
- Can't manage to get it run

Block Height: 15168460
Block hex:

Transactions (Etherscan): 294 transactions and 58 contract internal transactions in this block 

- Alchemy transfers API:
- internal: 58
- external: 143
- ERC20: 172
- ERC721: 20
- ERC1155: 2
- specialnft: 0
- All: 395

### Infura API

# Get price data for given date-range/block-range

In [ ]:

URL = f"https://api.coingecko.com/api/v3/ping"
r = requests.get(url = URL)
data = r.json()

print(data)

{'gecko_says': '(V3) To the Moon!'}


In [ ]:

URL = f"https://api.coingecko.com/api/v3/coins/list?include_platform=true"
r = requests.get(url = URL)
data = r.json()

print(data)
symbol_data = next(item for item in data if item["symbol"] == symbol)
id = symbol_data["id"]
id

{'status': {'error_code': 429, 'error_message': "You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits."}}


TypeError: string indices must be integers, not 'str'

In [ ]:
unix_end_date = 1554874500
URL = f"https://api.coingecko.com/api/v3/coins/{id}/market_chart/range?vs_currency=eur&from={unix_start_date}&to={unix_end_date}"
#params = {'vs_currency': 'eur', 'from': unix_start_date, 'to': unix_end_date}
r = requests.get(url = URL)
data = r.json()

print(data)


{'prices': [[1534896000000, 0.29101743227111937], [1534982400000, 0.27194256003119066], [1535068800000, 0.28334007331972505], [1535155200000, 0.2931066674907381], [1535241600000, 0.30424654164744885], [1535328000000, 0.3260326439748909], [1535414400000, 0.3759538339355391], [1535500800000, 0.39339740347120156], [1535587200000, 0.37590263951546904], [1535673600000, 0.41792134264988884], [1535760000000, 0.45634315293333516], [1535846400000, 0.4732703948921915], [1535932800000, 0.4547453923000906], [1536019200000, 0.44296724308966784], [1536105600000, 0.5096774165794833], [1536192000000, 0.40975696531990363], [1536278400000, 0.48152886475273127], [1536364800000, 0.4330458120091953], [1536451200000, 0.394879296292545], [1536537600000, 0.4006116109907809], [1536624000000, 0.37488486352805284], [1536710400000, 0.3237533485410619], [1536796800000, 0.3198782618698045], [1536883200000, 0.42645311771802713], [1536969600000, 0.39533873120835583], [1537056000000, 0.3973906946047553], [153714240000